In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
#Import .csv file, with correct encryption.

wellcome = pd.read_csv('WELLCOME_APCspend2013_forThinkful.csv', encoding = "ISO-8859-1")

# WELLCOME Data Cleaning

In [3]:
#Retitle
wellcome.columns = [c.replace(' ', '_') for c in wellcome.columns]
wellcome.rename(columns={'COST_(£)_charged_to_Wellcome_(inc_VAT_when_charged)': 'Cost', 'PMID/PMCID': 'PMID'}, inplace=True)

In [4]:
wellcome.head()

,PMID,Publisher,Journal_title,Article_title,Cost
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


## Identify the five most common journals and the total articles for each

From looking at the dataset, it seems that some of the journal titles have unnecessary whitespace, inconsistent capitalization from one entry to another, or subtitles referring to a particular issue of the journal. These subtitles tend to follow a ":" so I have chosen to split the titles at that point, and discard the second part of the title.

It is worth noting that a more complete cleaning would account for abbreviated titles and misspellings (e.g.: "JAIDS Journal of Acquired Immune Deficiency Syndrome" versus "Journal of Acquired Immune Deficiency Syndroms" versus "Journal of AIDS", all published through Wolters Kluwer.)

In [5]:
#Remove whitespace, capitalize:
wellcome['Journal_title'] = wellcome['Journal_title'].apply(lambda x: str.upper(str.lstrip(str.rstrip(str(x)))))

#Replace '&' with 'AND', unabbreviate 'J' to 'JOURNAL OF':
wellcome['Journal_title'] = wellcome['Journal_title'].apply(lambda x: x.replace('&', 'AND'))
wellcome['Journal_title'] = wellcome['Journal_title'].apply(lambda x: x.replace('J ', 'JOURNAL OF'))

#Strip remaining punctuation:
wellcome['Journal_title'] = wellcome['Journal_title'].apply(lambda x: x.split(':')[0])
wellcome['Journal_title'] = wellcome['Journal_title'].apply(lambda x: x.strip(',').strip('.'))

In [6]:
pd.DataFrame(wellcome['Journal_title'].value_counts()).head()

,Journal_title
PLOS ONE,190
JOURNAL OF BIOLOGICAL CHEMISTRY,53
NEUROIMAGE,34
NUCLEIC ACIDS RESEARCH,26
PLOS PATHOGENS,24


## Mean, median, standard deviation of the open-access cost per article for each journal.

In [7]:
#Remove '£' from decimal monetary values.

wellcome['Cost'] = wellcome['Cost'].apply(lambda x: x.strip('£'))

In [8]:
#Several values were in dollars, with the dollar sign after the number. We will convert these to pounds.
# https://www.irs.gov/individuals/international-taxpayers/yearly-average-currency-exchange-rates
# The 2013 yearly average currency conversion rate is: $1.00 = £0.665

def dollars_to_pounds(x):
    pattern1 = re.compile(r'(\d+(\.\d+)?\$)')
    if bool(pattern1.match(x)) == True:
        return round(float(pattern1.match(x).group().strip('$')) * 0.665, 2)
    else:
        return float(x)

In [9]:
wellcome['Cost'] = wellcome['Cost'].apply(lambda x: dollars_to_pounds(x))

In [10]:
#Reject extreme values:

wellcome['Cost'] = wellcome['Cost'].replace(999999.00, np.NaN)

mask = wellcome['Cost'] > 100000.00

wellcome.loc[mask, 'Cost'] = np.NaN

In [11]:
#Find standard deviation

journal_costs_std = wellcome.groupby('Journal_title').std()

In [12]:
journal_costs_meanmed = wellcome.groupby('Journal_title').aggregate([sum, np.mean, np.median])

In [13]:
#Setup dataframe to display descriptive statistics:
journal_costs = pd.DataFrame(wellcome['Journal_title'].value_counts())

journal_costs = journal_costs.merge(journal_costs_meanmed['Cost'], left_index=True, right_index=True)

journal_costs = journal_costs.merge(pd.DataFrame(journal_costs_std['Cost']), left_index=True,
                                    right_index=True).rename(columns = {'Journal_title': 'Count',
                                        'sum': 'Sum (£)', 'mean': 'Mean (£)', 'median': 'Median (£)', 'Cost': 'Standard Deviation (£)'})

journal_costs.head(20)

,Count,Sum (£),Mean (£),Median (£),Standard Deviation (£)
PLOS ONE,190,170107.36,934.655824,896.975,201.847589
JOURNAL OF BIOLOGICAL CHEMISTRY,53,74026.60,1423.588462,1301.140,411.954360
NEUROIMAGE,34,69725.71,2050.756176,2289.245,472.211498
NUCLEIC ACIDS RESEARCH,26,29874.00,1149.000000,852.000,442.940447
PLOS PATHOGENS,24,31538.45,1433.565909,1439.225,306.140757
PLOS GENETICS,24,36148.44,1643.110909,1712.730,153.366825
PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCES,22,17446.27,793.012273,742.045,464.269919
PLOS NEGLECTED TROPICAL DISEASES,20,31270.38,1563.519000,1516.115,156.521088
HUMAN MOLECULAR GENETICS,19,36738.75,2041.041667,2040.000,92.205662
NATURE COMMUNICATIONS,19,56414.40,3134.133333,3642.000,868.940927
